<a href="https://colab.research.google.com/github/akiran703/DQN_demo_for_SI/blob/main/DQN_for_spaceinvaders.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gym[atari]
!pip install autorom[accept-rom-license]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=446659 sha256=fd947e446a2ff27fdfaf5f627b726593ca4728eb0ed134a1547d01c78d9475f6
  Stored in directory: /root/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license


In [ ]:
import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
from gym import spaces

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from ale_py import ALEInterface
ale = ALEInterface()

from ale_py.roms import SpaceInvaders
ale.loadROM(SpaceInvaders)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#in order to reduce instability for when the Deep Q-learning is being trained, we apply a replay system to help.
#A replay system essentially is a way to store transistions that the agent observes and it can be used later training.
#This helps with reducing catostraphic forgetting in which the agent forgets all previous experiences as its new experience and reduces correlation between experiences.


#create a tuple to define the states
Transition = namedtuple('Transition',('state', 'action', 'next_state', 'reward'))

#creating that class that creates the Replay system
class ReplayMemory(object):
    #constructor creating a list to store the experiences
    def __init__(self, capacity):
        self.memory = []
        self.maxlen=capacity
        self.nextindex = 0

    # we can not keep every single transition, so we will remove the oldest one
    def push(self, state,action,reward,next_state,done):

      data = (state, action, reward, next_state, done)

      if self.nextindex >= len(self.memory):
        self.memory.append(data)
      else:
        self.memory[self.nextindex] = data
      self.nextindex = (self.nextindex + 1) % self.maxlen


    #encode the sample into arrays
    def encode_sample(self,indices):
      states, actions, rewards, next_states, dones = [], [], [], [], []
      for i in indices:
        d = self.memory[i]
        state, action, reward, next_state, done = d
        states.append(np.array(state,copy=False))
        actions.append(action)
        rewards.append(reward)
        next_states.append(next_state,copy=False)
        dones.append(done)
      return np.array(states), np.array(actions), np.array(rewards), np.array(next_states), np.array(dones)




    # getting a random experience
    def sample(self, batch_size):
      indx = np.random.randint(0, len(self.memory) - 1, size = batch_size)
      return self.encode_sample(indx)


    # the length of the memory
    def __len__(self):
        return len(self.memory)


In [ ]:
class DQN(nn.Module):
  def __init__(self,observation_space: spaces.Box, action_space: spaces.discrete) -> None:
     super().__init__()
     # checks to enable that the observation space is a box, it forms a channels x widith x height, the action space is discrete
     assert type(observation_space) == spaces.Box
     assert type(observation_space.shape) == 3
     assert type(action_space) == spaces.Discrete

      #Convolution layer with activation function
     self.conv = nn.Sequential(
         nn.Conv2d(in_channels=observation_space.shape[0], out_channels=32, kernel_size=8, stride=4),
         nn.ReLU(),
         nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
         nn.ReLU(),
         nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
         nn.ReLU()
     )

     #fully connected layer
     self.fc = nn.Sequential(
            nn.Linear(in_features=64*7*7 , out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=action_space.n)
        )
  #feed foward neural network
  def forward(self, x):
        conv_out = self.conv(x).view(x.size()[0],-1)
        return self.fc(conv_out)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
#function to create an environment

def create_env():
  base_env = gym.make("ALE/SpaceInvaders-v5")
  env = gym.wrappers.GrayScaleObservation(base_env)
  env = gym.wrappers.FrameStack(env,4)
  env = gym.wrappers.ResizeObservation(env, (84,84))
  #print(base_env.action_space)
  #print(env.observation_space)

  return env




real_env = create_env()

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
